# Drug ATC Classification

Creating the Drug-Atc pair dataset

In [ ]:
import pandas as pd

input_file = 'Data/chembl/drugs.csv'
output_file = 'Dataset/drug-atc/dataset.csv'
atc_mapping_file = 'Dataset/atc/dataset.csv'

df = pd.read_csv(input_file, sep=';', encoding='utf-8')

df_subset = df[["Smiles", "Level 1 ATC Codes"]].copy()

df_subset.dropna(inplace=True)

df_exploded = df_subset.assign(
    atc_level1=df_subset["Level 1 ATC Codes"].str.split(" \| ")
).explode("atc_level1")

final_df = df_exploded[["Smiles", "atc_level1"]].rename(columns={"Smiles": "smiles"})

unique_atcs = sorted(final_df["atc_level1"].unique())
atc_to_num = {atc: idx for idx, atc in enumerate(unique_atcs)}
final_df["atc_numeric"] = final_df["atc_level1"].map(atc_to_num)

final_df = final_df[~final_df['atc_level1'].str.contains("V - VARIOUS")]

final_df = final_df[~final_df["smiles"].str.contains(r"(?:Cl|Na|HCl|Br|SO4)", regex=True)]

counts = final_df["atc_level1"].value_counts()
valid_classes = counts[counts > 1].index
final_df = final_df[final_df["atc_level1"].isin(valid_classes)]

final_df.to_csv(output_file, index=False)

used_atcs = final_df["atc_level1"].unique()
atc_mapping_df = pd.DataFrame({
    "atc_level1": used_atcs,
    "atc_numeric": [atc_to_num[a] for a in used_atcs]
}).set_index("atc_level1")
atc_mapping_df.to_csv(atc_mapping_file)

print(f"✅ Saved: {output_file} and {atc_mapping_file}")

print(f"Final dataset size: {len(final_df)} samples, {len(valid_classes)} ATC classes")



✅ Saved: Dataset/drug-atc/dataset.csv and Dataset/atc/dataset.csv
Final dataset size: 2053 samples, 13 ATC classes


## Final Dataset

Extracting ECFP4 Features

In [ ]:
import pandas as pd, numpy as np
from rdkit import Chem
from rdkit.Chem import MACCSkeys
import rdkit.DataStructs as ds
from rdkit.Chem.rdFingerprintGenerator import GetMorganGenerator
from Mold2_pywrapper import Mold2

input_file   = "Dataset/drug-atc/dataset.csv"
output_file  = "Dataset/final/dataset.csv"

ecfp_bits  = 2048

def fp_to_array(fp, n_bits):
    arr = np.zeros((n_bits,), dtype=int)
    ds.ConvertToNumpyArray(fp, arr)
    return arr

raw_df = pd.read_csv(input_file, dtype={"atc_numeric": int})
raw_df.dropna(subset=["smiles"], inplace=True)
raw_df = raw_df[~raw_df["smiles"].str.contains(r"\.", regex=True)]
raw_df.reset_index(drop=True, inplace=True)

valid_idx, mols = [], []
for idx, smi in enumerate(raw_df["smiles"]):
    m = Chem.MolFromSmiles(smi)
    if m:
        valid_idx.append(idx)
        mols.append(m)

if not mols:
    raise ValueError("No valid SMILES found!")

core_df = raw_df.loc[valid_idx].reset_index(drop=True)

print("🧬 Calculating ECFP4 …")
morgan_gen = GetMorganGenerator(
    radius=2, fpSize=ecfp_bits,
    includeChirality=False, useBondTypes=True
)
ecfp_mat = [fp_to_array(morgan_gen.GetFingerprint(m), ecfp_bits) for m in mols]
ecfp_df  = pd.DataFrame(ecfp_mat, columns=[f"ECFP4_{i}" for i in range(ecfp_bits)])

features_df = pd.DataFrame(ecfp_df)

features_df["label"] = core_df["atc_numeric"].values

features_df.replace([np.inf, -np.inf], np.nan, inplace=True)
features_df.dropna(inplace=True)

features_df.to_csv(output_file, index=False)
print(f"✅ Saved final dataset {features_df.shape} → {output_file}")


c:\Users\jyoti\Anaconda3\envs\drug-detection\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


🧬 Calculating ECFP4 …
✅ Saved final dataset (2010, 2049) → Dataset/final/dataset.csv
